In [ ]:
import os
import re

import certifi
import nltk
import pandas as pd
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from nltk.corpus import stopwords
from scipy.cluster import hierarchy as sch
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [1]:
# Set the environment variables within your Python script
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

print("REQUESTS_CA_BUNDLE:", os.environ.get("REQUESTS_CA_BUNDLE"))
print("SSL_CERT_FILE:", os.environ.get("SSL_CERT_FILE"))

## was gna kill myself over this ERROR BRUH

REQUESTS_CA_BUNDLE: /Users/sarahSTACS/DSA4264-Detoxify/penv/lib/python3.12/site-packages/certifi/cacert.pem
SSL_CERT_FILE: /Users/sarahSTACS/DSA4264-Detoxify/penv/lib/python3.12/site-packages/certifi/cacert.pem


Separate data by years

In [91]:
data_path2 = "/Users/sarahSTACS/Downloads/final_data.csv"
df = pd.read_csv(data_path2)
print(df.columns)
# filtered_df = df[["text", "gold_label"]][df["year"] == 2023]
# filtered_df.shape[0]

Index(['text', 'timestamp', 'username', 'link', 'link_id', 'parent_id', 'id',
       'subreddit_id', 'moderation', 'year', 'word_count', 'Sensitive Group',
       'Classification'],
      dtype='object')


get count of each unique value of classification 

In [3]:
classification_counts = df["Classification"].value_counts()
print(classification_counts)

Classification
No Hate/Toxic    355079
Hate 1            22641
Toxic 1           15097
Toxic 2            5563
Hate 2              967
Toxic 3             432
Hate 3              221
Name: count, dtype: int64


clean data which might not be needed anymore

In [4]:
filtered_df = df[df["Classification"] != "No Hate/Toxic"]
text = filtered_df["text"].tolist()
len(text)

44921

In [5]:
# Custom dictionary for translating Singlish terms to standard English
singlish_dict = {
    "lah": "",
    "lor": "",
    "sia": "",
    "leh": "",
    "fuck": "",
    "shit": "",
    "fucker": "",
    "bitch": "",
    "fuk": "",
    "cunt" "fucking": "",
    "sian": "bored",
    "lepak": "relax",
    "shiok": "great",
    "kiasu": "fear of missing out",
    "fomo": "fear of missing out",
    "kopi": "coffee",
    "yaya papaya": "arrogant person",
    "cheem": "complex",
    "aiyo": "oh no",
    "dabao": "takeaway",
    "paiseh": "embarrassed",
    "koped": "stolen",
    "gahmen": "government",
    "gahment": "government",
    "liao": "already",
    "siao": "crazy",
}

# Get the default NLTK English stop words
nltk_stop_words = stopwords.words("english")
custom_stop_words = ["hes", "shes"]  # Add any additional custom stop words

# Combine NLTK stop words with custom stop words
combined_stop_words = set(nltk_stop_words + custom_stop_words)


def clean_singlish(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove extra spaces and special characters
    text = re.sub(r"[^\w\s]", "", text)

    # Tokenize the sentence into words
    words = text.split()

    # Translate Singlish words to Standard English
    cleaned_words = [singlish_dict.get(word, word) for word in words]

    # Remove stop words
    cleaned_words = [word for word in cleaned_words if word not in combined_stop_words]

    # Join the words back into a cleaned sentence
    cleaned_text = " ".join(cleaned_words)

    # Remove any leftover empty phrases
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()

    return cleaned_text


filtered_df["clean_text"] = filtered_df["text"].apply(clean_singlish)
print(filtered_df[["text", "clean_text"]].head())
filtered_df.shape
filtered_df.head()
text = filtered_df["clean_text"].tolist()

                                                 text  \
15                                Tell The F Mon G3rs   
29  Don't ask so many questions, if not PAP arrest...   
45  It's a claim that is well supported by various...   
50  Why come all the way here and complain dipshit...   
74  I can understand giving exemptions for people ...   

                                           clean_text  
15                                    tell f mon g3rs  
29  dont ask many questions pap arrest seditious s...  
45  claim well supported various studies blm hand ...  
50  come way complain dipshit pajeet ape enjoy lif...  
74  understand giving exemptions people cannot vac...  


/var/folders/nh/xyplbgbn1wlc853r_mjxqg8h0000gq/T/ipykernel_11723/725180145.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["clean_text"] = filtered_df["text"].apply(clean_singlish)


TOPIC MODEL CUSTOM

In [8]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForMaskedLM, AutoTokenizer
from umap import UMAP

# Initialize models for BERTopic
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
umap_model = UMAP(
    n_neighbors=30, n_components=3, min_dist=0.11, metric="cosine", random_state=42
)
hdbscan_model = HDBSCAN(
    min_cluster_size=25,
    min_samples=1,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)
representation_model = KeyBERTInspired()

# Train the BERTopic model
topic_model_custom = BERTopic(
    embedding_model=sentence_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
)

# Assuming 'text' is a pre-defined list of documents
topics, probs = topic_model_custom.fit_transform(text)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

In [64]:
from scipy.cluster import hierarchy as sch

#  Hierarchical topics
linkage_function = lambda x: sch.linkage(
    x, "complete", optimal_ordering=True
)  # use either complete or ward
hierarchical_topics_1 = topic_model_1.hierarchical_topics(
    text, linkage_function=linkage_function
)
topic_model_custom.visualize_hierarchy(hierarchical_topics=hierarchical_topics_1)

100%|██████████| 360/360 [01:22<00:00,  4.38it/s]


Before merging

In [75]:
topic_keywords = {}
for topic_num in sorted(topic_model_custom.get_topics()):
    try:
        words, scores = zip(*topic_model_custom.get_topic(topic_num))
        topic_keywords[topic_num] = ", ".join(words)
        print(f"Topic {topic_num}: {', '.join(words)}")
    except ValueError:
        print(f"No words found for Topic {topic_num}, skipping...")
        topic_keywords[topic_num] = "No relevant words"  # Handling topics without words

Topic -1: singaporean, singapore, race, countries, cant, country, cannot, better, lol, kids
Topic 0: chinese, china, chinas, nonchinese, mandarin, taiwanese, taiwan, chinatown, singaporean, singaporeans
Topic 1: muslims, christians, muslim, islamic, christianity, islam, christian, evangelicals, atheists, evangelical
Topic 2: singaporeans, singaporean, singapores, singapore, foreigners, foreigner, countries, foreign, migrant, country
Topic 3: foreigners, foreigner, immigrants, foreign, immigrant, overseas, migrants, immigration, deported, migrant
Topic 4: feminism, sexism, feminists, feminist, sexist, misogynistic, genders, patriarchy, masculinity, gender
Topic 5: racist, racists, racism, racial, racially, interracial, discriminate, race, races, discrimination
Topic 6: gay, gays, homosexuals, homosexual, homos, homophobe, homophobic, homosexuality, homophobia, homophobes
Topic 7: boomers, boomer, millennials, millenials, millennial, millenial, generations, generation, youngsters, uncles

In [109]:
topics_to_merge = [
    [232, 9],  # crime
    [221, 0],  # rasim/natioanlity
    [217, 2],  # religion
    [4, 20, 185, 186],
    [194, 11],
    [9, 233],
]  # gender

topic_model_custom.merge_topics(text, topics_to_merge)

After merging

In [110]:
topic_keywords = {}
for topic_num in sorted(topic_model_custom.get_topics()):
    try:
        words, scores = zip(*topic_model_custom.get_topic(topic_num))
        topic_keywords[topic_num] = ", ".join(words)
        print(f"Topic {topic_num}: {', '.join(words)}")
    except ValueError:
        print(f"No words found for Topic {topic_num}, skipping...")
        topic_keywords[topic_num] = "No relevant words"

Topic -1: singaporean, singapore, race, countries, cant, country, cannot, better, lol, kids
Topic 0: racist, racists, racism, racial, xenophobic, xenophobia, black, discrimination, race, malaysians
Topic 1: chinese, china, chinas, nonchinese, taiwanese, mandarin, taiwan, chinatown, singaporean, singaporeans
Topic 2: muslims, christians, christianity, atheists, muslim, atheist, christian, islamic, islam, religion
Topic 3: singaporean, singaporeans, singapores, singapore, foreigners, foreigner, foreign, countries, citizen, migrant
Topic 4: feminism, feminists, sexism, feminist, sexist, misogynistic, patriarchy, genders, equality, masculinity
Topic 5: boomers, millennials, boomer, millenials, millennial, generations, generation, seniors, elderly, elders
Topic 6: homosexuals, homosexuality, heterosexuals, gays, homophobes, homophobe, homophobia, homosexual, homophobic, gay
Topic 7: vaccines, vaccinations, vaccination, vaccinate, vaccine, vaccinated, masks, mask, maskers, unvaccinated
Topic

In [104]:
filtered_df["custom_topic"] = topics
topic_classification_counts_custom = (
    filtered_df.groupby(["custom_topic", "Classification"]).size().unstack(fill_value=0)
)
topic_classification_counts_custom["custom_Total"] = (
    topic_classification_counts_custom.sum(axis=1)
)

# Print the resulting DataFrame
print(topic_classification_counts_custom)
topic_classification_counts_custom.to_csv(
    "Topic_Classification_Counts_custom.csv", index=True
)
tc_df_custom = pd.DataFrame(topic_classification_counts_custom)
tc_df_custom["Combined Hate & Toxic 3"] = tc_df_custom["Hate 3"] + tc_df["Toxic 3"]
sorted_df_custom = tc_df_custom.sort_values(
    by="Combined Hate & Toxic 3", ascending=False
)

# Find the top 3 values (this includes handling ties)
top_values = sorted_df_custom["Combined Hate & Toxic 3"].nlargest(3)
min_top_value = top_values.min()  # Get the minimum value of the top 3

# Filter the DataFrame to include all topics with values equal to the third highest
top_topics_df_custom = sorted_df_custom[
    sorted_df_custom["Combined Hate & Toxic 3"] >= min_top_value
]

print("Filtered DataFrame for Top 'Combined Hate & Toxic 3' Topics (including ties):")
print(top_topics_df_custom)

Classification  Hate 1  Hate 2  Hate 3  Toxic 1  Toxic 2  Toxic 3  \
custom_topic                                                        
-1                6799     311      98     5802     2179      190   
 0                2606     105       4       79       24        2   
 1                1446      49       5       91       56        0   
 2                1145      26       1      279       93        2   
 3                 866      38       0       76       11        0   
...                ...     ...     ...      ...      ...      ...   
 268                 2       0       0       11        9        3   
 269                 8       1       0       12        4        0   
 270                 4       0       0       16        4        1   
 271                 7       0       0       14        4        0   
 272                14       0       0        9        2        0   

Classification  custom_Total  
custom_topic                  
-1                     15379  
 0       

In [76]:
filtered_df["custom_topic"] = topics
filtered_df["Topic_Words"] = filtered_df["custom_topic"].map(topic_keywords)
filtered_df.head()

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation,year,word_count,Sensitive Group,Classification,clean_text,custom_topic,Topic_Words
15,Tell The F Mon G3rs,16/6/20 9:22,revisedchampion,/r/singapore/comments/ha0jfx/better_suggestion...,t3_ha0jfx,t1_fuzu5ag,fuzu66z,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",2020,5.0,No,Toxic 2,tell f mon g3rs,82,"hush, shut, singh, geh, sh, hannah, koh, brah,..."
29,"Don't ask so many questions, if not PAP arrest...",2023-10-10 07:15:37,RE_TrollzL0rd,/r/singaporehappenings/comments/173vano/privat...,t3_173vano,t1_k45ajnz,k48tp91,t5_70s6ew,"{'banned_at_utc': None, 'mod_reason_by': None,...",2023,20.0,No,Toxic 1,dont ask many questions pap arrest seditious s...,15,"voting, voters, vote, ballot, ballots, votes, ..."
45,It's a claim that is well supported by various...,6/8/2020 11:37,Talkingtomytoilet,/r/singapore/comments/gyxf42/the_myth_of_syste...,t3_gyxf42,t1_ftcyjev,ftczbxh,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",2020,21.0,Yes,Hate 1,claim well supported various studies blm hand ...,-1,"singaporean, singapore, race, countries, cant,..."
50,Why come all the way here and complain dipshit...,20/2/2022 2:21,viviseca,/r/SingaporeRaw/comments/swdnv3/its_funny_beca...,t3_swdnv3,t1_hxn3238,hxnl9cs,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",2022,33.0,Yes,Hate 2,come way complain dipshit pajeet ape enjoy lif...,-1,"singaporean, singapore, race, countries, cant,..."
74,I can understand giving exemptions for people ...,2/2/22 9:39,Silverelfz,/r/singapore/comments/siafpw/muslims_unvaccina...,t3_siafpw,t1_hv8hm2o,hv992pf,t5_2qh8c,"{'controversiality': 0, 'collapsed_reason_code...",2022,31.0,Yes,Hate 1,understand giving exemptions people cannot vac...,13,"vaccinations, vaccines, vaccination, vaccine, ..."


In [77]:
filtered_df.to_csv("full_data_with_topic.csv", index=False)
# Filter the DataFrame
df_topic_negative_one = filtered_df[filtered_df["custom_topic"] == -1]

# Save the filtered DataFrame to CSV
df_topic_negative_one.to_csv("full_data_with_topic_-1.csv", index=False)

update topic model custom after merging

In [153]:
updated_topics, _ = topic_model_custom.transform(text)

In [177]:
info_df = topic_model_custom.get_topic_info()
info_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,15379,-1_singaporean_singapore_race_countries,"[singaporean, singapore, race, countries, cant, country, cannot, better, lol, kids]","[well said people calling racist never experienced probably theyre chinese asians happens like us often compared indians malays, want children prefer kind cannot yea dude theres million aspects person consider like someone could hot smart sweet hell means jack cause race huh see power power, im sure overseas uni way way worse singapore small better police schools well dont make race whos worst]"
1,0,6259,0_chinese_china_mandarin_nonchinese,"[chinese, china, mandarin, nonchinese, singaporean, racists, malaysians, taiwanese, racist, taiwan]","[racist race lol, malay chinese country, dumb comment singapore majority chinese like us doesnt mean singaporeans must look like chinese every race looks singaporean singapore]"
2,1,1675,1_religion_religious_religions_islam,"[religion, religious, religions, islam, muslim, muslims, islamic, atheist, atheists, christianity]","[brain religion, brain religion, could even religious thing]"
3,2,1546,2_singaporeans_singaporean_singapores_singapore,"[singaporeans, singaporean, singapores, singapore, foreigners, foreigner, countries, foreign, country, citizen]","[show us singaporean jobs malaysian singapore u, yes singaporeans dont say guy, singaporeans want meritocracy work want people question singaporeans singaporeans]"
4,3,1203,3_sexism_feminism_sexist_feminists,"[sexism, feminism, sexist, feminists, feminist, misogynistic, masculinity, genders, gender, patriarchy]","[women dont need good im saying women compete men women play games etc mens leagues taken seriously isnt men stronger women lmfao, youre looking gender equality singapore might find ns wont find, pretty sure men job women dont want ns]"
...,...,...,...,...,...
231,230,25,230_blaming_victim_victims_pity,"[blaming, victim, victims, pity, complaining, accidents, harassment, embarrassed, bully, forgive]","[yeah victim blaming upvoted much, hey yo victim blaming, nice victim blaming thread]"
232,231,25,231_scammers_urgh_scam_refund,"[scammers, urgh, scam, refund, bankrupt, shithole, scums, harass, shout, cushysounding]","[speak softly bait increase volume shout mercy scums tricks ppl hard earn money, yep haha made noise head office agreed refund deposit wanna vent encountering unethical practice fucking person spend 5k vacuum la oh no didnt even think call happened urgh, overground okay since gonna build inevitable wall segregate yishun anywayand get yishunites pay]"
233,232,25,232_soap_dishwasher_wash_launder,"[soap, dishwasher, wash, launder, wipe, dont, dirty, spraying, laundry, drop]","[dont pick soap, dont drop soap officer, dont drop soap guys around]"
234,233,25,233_girl_girls_prostitute_girlfriends,"[girl, girls, prostitute, girlfriends, whore, mothersistergirlfriend, swallow, cringy, friends, loudly]","[close midnight train circle line thrashy girl talking loudly friend behaving like hot stuff truth nobody gave shyt started worming floor unclothed, start girl calling another girl prostitute, girls called zi hui must panicking girls named zi hui dating jayden gg]"


In [173]:
from scipy.cluster import hierarchy as sch

#  Hierarchical topics
linkage_function = lambda x: sch.linkage(
    x, "complete", optimal_ordering=True
)  # use either complete or ward
hierarchical_topics_1 = topic_model_custom.hierarchical_topics(
    text, linkage_function=linkage_function
)
topic_model_custom.visualize_hierarchy(hierarchical_topics=hierarchical_topics_1)

100%|██████████| 234/234 [00:49<00:00,  4.71it/s]


In [154]:
final_topic_keywords = {}

# Assuming 'topic_model_custom' is your topic model object
for topic_num in sorted(topic_model_custom.get_topics()):
    try:
        words, _ = zip(*topic_model_custom.get_topic(topic_num))
        final_topic_keywords[topic_num] = ", ".join(words)
    except ValueError:
        # Handle topics without keywords (if any)
        final_topic_keywords[topic_num] = "No relevant words"

# Optionally print the keywords to verify
for topic_num, keywords in final_topic_keywords.items():
    print(f"Topic {topic_num}: {keywords}")

Topic -1: singaporean, singapore, race, countries, cant, country, cannot, better, lol, kids
Topic 0: chinese, china, mandarin, nonchinese, singaporean, racists, malaysians, taiwanese, racist, taiwan
Topic 1: religion, religious, religions, islam, muslim, muslims, islamic, atheist, atheists, christianity
Topic 2: singaporeans, singaporean, singapores, singapore, foreigners, foreigner, countries, foreign, country, citizen
Topic 3: sexism, feminism, sexist, feminists, feminist, misogynistic, masculinity, genders, gender, patriarchy
Topic 4: gays, gay, homosexuals, homos, homophobe, homophobes, homosexual, heterosexuals, homophobic, homophobia
Topic 5: boomers, millennials, boomer, millenials, millennial, generations, generation, seniors, elderly, elders
Topic 6: rape, rapist, rapists, raped, raping, molested, pedophile, pedophiles, crimes, jail
Topic 7: vaccines, vaccinations, vaccination, vaccinate, vaccine, vaccinated, masks, mask, maskers, unvaccinated
Topic 8: motorcyclists, cyclists,

In [155]:
filtered_df["Updated_Topic"] = updated_topics

In [156]:
def get_keywords_for_topic(topic):
    return final_topic_keywords.get(topic, "No relevant words")


# Map the updated topics to keywords
filtered_df["Final Topic Keywords"] = filtered_df["Updated_Topic"].apply(
    get_keywords_for_topic
)

In [158]:
# Checking for any missing keywords entries
missing_keywords_count = (
    filtered_df["Final Topic Keywords"] == "No relevant words"
).sum()
print(f"Count of entries with missing keywords: {missing_keywords_count}")

# Display rows with missing keywords to understand the issue
print(filtered_df[filtered_df["Final Topic Keywords"] == "Keywords missing"])

Count of entries with missing keywords: 0
Empty DataFrame
Columns: [text, timestamp, username, link, link_id, parent_id, id, subreddit_id, moderation, year, word_count, Sensitive Group, Classification, clean_text, custom_topic, Topic_Words, Updated_Topic, Final Topic, Final Topic Name, Final Topic Keywords]
Index: []


In [164]:
topic_mapping = {
    0: (1, "Racism"),
    1: (2, "Religion"),
    2: (1, "Racism"),
    3: (7, "Gender"),
    4: (4, "LGBTQ+"),
    5: (3, "Generational"),
    6: (9, "Crimes"),
    7: (6, "COVID-19"),
    8: (11, "Transportation"),
    9: (5, "Work"),
    10: (8, "Government"),
    11: (13, "Body Image"),
    13: (10, "Housing"),
    16: (12, "Education"),
}


def map_final_topics(updated_topic):
    if updated_topic in topic_mapping:
        final_topic_num, final_topic_name = topic_mapping[updated_topic]
        return final_topic_num, final_topic_name
    else:
        return None, None  # Return None for topics that do not map


# Apply the mapping to create new columns for Final Topic and Final Topic Name
filtered_df["Final Topic"], filtered_df["Final Topic Name"] = zip(
    *filtered_df["Updated_Topic"].apply(map_final_topics).tolist()
)
filtered_df["Final Topic"] = filtered_df["Final Topic"].astype("Int64")

# Display the DataFrame to verify the results
print(filtered_df[["Updated_Topic", "Final Topic", "Final Topic Name"]])

        Updated_Topic  Final Topic Final Topic Name
15                 66         <NA>             None
29                 10            8       Government
45                 -1         <NA>             None
50                 -1         <NA>             None
74                  7            6         COVID-19
...               ...          ...              ...
399963             -1         <NA>             None
399964              0            1           Racism
399978             -1         <NA>             None
399988              2            1           Racism
399990              2            1           Racism

[44921 rows x 3 columns]


In [165]:
filtered_df.to_csv("final_topics_redistribute.csv", index=False)

In [89]:
existing_df = pd.read_csv("final_topics_redistribute.csv")
neg_one_df = pd.read_csv(
    "/Users/sarahSTACS/Downloads/Telegram Lite/topics_-1_redistribute.csv"
)

In [90]:
print("Duplicate 'text' entries in final_df:")
print(existing_df[existing_df["text"].duplicated(keep=False)])

Duplicate 'text' entries in final_df:
                                                    text            timestamp  \
3565                         You are a retard fat cibai.  2021-07-18 17:07:21   
3593                                   I bet u look dyel       20/1/2022 7:11   
3644                              How about you be a man       4/17/2023 8:44   
3686             there are white people outside my house        31/7/22 12:31   
3827   Congrats for what You mean one less foreign th...       21/8/2023 5:07   
3987   So tilted you have decided to stalk me Ask me ...      22/6/2023 14:34   
4183   You gonna cry little pathetic abandoned virgin :D       8/1/2022 10:20   
5218   You gonna cry little pathetic abandoned virgin :D       8/1/2022 10:21   
5492   And here I thought city folks are educated. Bu...         19/4/20 1:59   
6448   Whats the difference between a police officer ...       20/9/2023 8:42   
8188                      This is your brain on religion      3/26/2022

In [77]:
# Load existing CSV file
existing_df = pd.read_csv(
    "/Users/sarahSTACS/DSA4264-Detoxify/model-2/final_topics_no_-1_redistribute.csv"
)
neg_one_df = pd.read_csv(
    "/Users/sarahSTACS/Downloads/Telegram Lite/topics_-1_redistribute.csv"
)

# Ensure that 'filtered_df' contains all necessary columns
if "Updated_Topic" not in neg_one_df.columns:
    filtered_df["updated_Topic"] = (
        None  # Adjust according to how you want to handle missing data
    )

if "Final Topic Keywords" not in neg_one_df.columns:
    filtered_df["Final Topic Keywords"] = (
        None  # Adjust according to how you want to handle missing data
    )

# Append 'filtered_df' to 'existing_df'
combined_final_df = pd.concat([existing_df, neg_one_df], ignore_index=True)

# Replace -1.0 in 'Final Topic' with NaN and 'Outlier' in 'Final Topic Name' with NaN
combined_final_df["Final Topic"] = (
    combined_final_df["Final Topic"].replace(-1.0, pd.NA).astype("Int64")
)
combined_final_df["Final Topic Name"] = combined_final_df["Final Topic Name"].replace(
    "Outlier", pd.NA
)
combined_final_df["Updated_Topic"] = combined_final_df["Updated_Topic"].astype("Int64")
# Save the combined DataFrame to a new CSV file
final_topic_df = combined_final_df.copy()
final_topic_df.drop(
    columns=["Updated_Topic", "Final Topic Keywords"], errors="ignore", inplace=True
)

# Save the adjusted combined DataFrame to a new CSV file
final_topic_df.to_csv("final_topic_df.csv", index=False)

In [78]:
num_duplicates = final_topic_df.duplicated().sum()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [166]:
filtered_df = filtered_df[filtered_df["custom_topic"] != -1]

# Save the filtered DataFrame to a CSV file
filtered_df.to_csv("final_topics_no_-1_redistribute.csv", index=False)

merging df for original and -1 topic

In [127]:
final_df = pd.read_csv("/Users/sarahSTACS/DSA4264-Detoxify/model-2/final_topic_df.csv")
neg_one_df = pd.read_csv("/Users/sarahSTACS/DSA4264-Detoxify/model-2/topic_-1_rep.csv")
# Function to clean topic words by removing numbers and underscores


def clean_topic_words(topic):
    # Remove leading -number_ or number_, then replace underscores with commas
    cleaned_topic = re.sub(r"^[-]?\d+_", "", topic)
    return cleaned_topic.replace("_", ", ")


# Apply the cleaning function to the 'Topic Words' column in neg_one_df
neg_one_df["Topic Words"] = neg_one_df["Topic Words"].apply(clean_topic_words)
neg_one_df = neg_one_df.loc[:, ~neg_one_df.columns.str.contains("^Unnamed")]
num_duplicates = final_df.duplicated().sum()
final_df = final_df.drop_duplicates()
# Print the number of duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
filtered_no_neg_one = final_df[final_df["custom_topic"] != -1]
neg_one_df_for_append = (
    neg_one_df.drop(columns=["Topic_Words"])
    .rename(columns={"Topic Words": "Topic_Words"})
    .drop(columns=["Title_Words", "title_words_str", "Topic Representation"])
)
# Append neg_one_df to final_df
combined_df = pd.concat([filtered_no_neg_one, neg_one_df_for_append], ignore_index=True)
combined_df = combined_df.rename(columns={"Topic": "Topic Num"})
combined_df["Final Topic Name"] = combined_df["Final Topic Name"].replace(
    "Romanian", "Racism"
)
combined_df.to_csv("combined_df.csv", index=False)

Number of duplicate rows: 0
